In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

from pprint import pprint
from zen_garden.postprocess.results import Results

### Working with results of a ZenGarden run

This notebook shows some examples on how to handle the results of a `zen-garden` run. It uses the outputs of the tests as examples. Therefore, you need to run the tests via 
```
coverage run --source="zen_garden" -m pytest -v tests/testcases/run_test.py
```
to get the data.

### Reading in the results

The results can be read in using the `Results` class. This will automatically read in all relevant files, independent of the number of scenarios or if the optimization was run with perfect or myopic foresight. However, you should know what type of input you are reading in, as the output of some routines will depend on the type.

In [40]:
# This has no scenarios and perfect foresight
res_basic = Results("../outputs/test_1a/")
# Here we have three different scenarios
res_scenario = Results("../outputs/test_6a/")
# This was run with myopic foresight
res_mf = Results("../outputs/test_4g/")

### Accessing the config

The standard dictionaries, i.e. system, analysis, scenarios, solver and dictSequenceTimeSteps can be accessed via the results dict.

In [41]:
# Show the analysis settings
pprint(res_basic.results["analysis"])

{'dataset': 'C:\\Users\\janis\\Dropbox\\SIS\\Subscriptions\\Sansavini\\ZEN-garden\\tests\\testcases\\test_1a',
 'discountRate': 0.06,
 'fileFormat': 'csv',
 'headerDataInputs': {'setCarriers': ['demandCarrier',
                                      'availabilityCarrier',
                                      'exportPriceCarrier',
                                      'importPriceCarrier'],
                      'setConversionTechnologies': ['availability'],
                      'setEdges': ['edge'],
                      'setExistingTechnologies': ['existingTechnology'],
                      'setNodes': ['node', 'x', 'y'],
                      'setScenarios': ['scenario'],
                      'setTimeSteps': ['time'],
                      'setTimeStepsYearly': ['year'],
                      'setTransportTechnologies': ['availability',
                                                   'costPerDistance',
                                                   'distanceEuclidean',
    

### Extracting individual data frames

You can access individual data frames from any component with the `get_dataframe` method. However, the behavior differs slightly for the usecases.
If the optimization was run with perfect foresight and no scenarios, then the data frame is returned as `pandas.Series`. For myopic foresight, the final dataframe is stiched together from the relevant outputs if the component is set yearly. If multiple scenarios were run, a dictionary containing the data frame for all different scenarios is returned. However, you can specify the scenario in advance, then only a single data frame is returned.

Additionally, the method provides a shortcut to save the extracted dataframe directly to a CSV, or in case of different scenarios to a bundle of CSV files.

In [42]:
# This will return a single data frame
df = res_basic.get_dataframe('exportCarrierFlow')
df

heat         CH  0    0.0
             DE  0    0.0
natural_gas  CH  0    0.0
             DE  0    0.0
Name: exportCarrierFlow, dtype: float64

In [43]:
# you can also save the dataframe directly to a csv:
df = res_basic.get_dataframe('carbonEmissionsOvershoot', to_csv="test")
# this prints out the content of the generated csv
!cat test.csv

,carbonEmissionsOvershoot
0,0.0


In [44]:
# if we have multiple scenarios, all dataframe will be returned in a dict
df_dict = res_scenario.get_dataframe('exportCarrierFlow')
pprint(df_dict)

{'scenario_': heat         CH  0    0.0
             DE  0    0.0
natural_gas  CH  0    0.0
             DE  0    0.0
Name: exportCarrierFlow, dtype: float64,
 'scenario_2': heat         CH  0    0.0
             DE  0    0.0
natural_gas  CH  0    0.0
             DE  0    0.0
Name: exportCarrierFlow, dtype: float64,
 'scenario_3': heat         CH  0    0.0
             DE  0    0.0
natural_gas  CH  0    0.0
             DE  0    0.0
Name: exportCarrierFlow, dtype: float64}


In [45]:
# saving this to a csv will create a file for each scenario
df_dict = res_scenario.get_dataframe('exportCarrierFlow', to_csv="test2")
# list files and print content of one
!echo "Generated files: "
!ls test2*
!echo 
!echo "Output of test2_scenario_2.csv"
!cat test2_scenario_2.csv

Generated files: 
test2_scenario_.csv  test2_scenario_2.csv  test2_scenario_3.csv

Output of test2_scenario_2.csv
,,,exportCarrierFlow
heat,CH,0,0.0
heat,DE,0,0.0
natural_gas,CH,0,0.0
natural_gas,DE,0,0.0


In [46]:
# we can also specify a specific scneario
df = res_scenario.get_dataframe('exportCarrierFlow', scenario="scenario_2")
df

heat         CH  0    0.0
             DE  0    0.0
natural_gas  CH  0    0.0
             DE  0    0.0
Name: exportCarrierFlow, dtype: float64

In [48]:
# This will collect the values from the results folders of the different myopic foresight runs
df = res_mf.get_dataframe("carbonEmissionsCumulative")
df

0    149.341143
1    298.682286
2    448.023428
Name: carbonEmissionsCumulative, dtype: float64

### Helper functions

The `Results` class implements some helper functions. The function `calculateTotalValue` allows you to calculate the total Value of a component, for a given year, element and scenario if desired. And the function `calculateFullTimeSeries` calculates the full (disaggregate) time series for a given component.

In [49]:
# We read out a normal dataframe
df = res_mf.get_dataframe("inputFlow")
print(df)

natural_gas_boiler  natural_gas  CH  0       1.356468
                                     1       1.356468
                                     2       1.356468
                                     3      24.427713
                                     4      24.427713
                                     5      24.427713
                                     6      10.719171
                                     7      10.719171
                                     8      10.719171
                                     9      13.774994
                                     10     13.774994
                                     11     13.774994
                                     12     31.583558
                                     13     31.583558
                                     14     31.583558
                                     15     27.856524
                                     16     27.856524
                                     17     27.856524
                            

In [50]:
# We can calculate the total value of the dataframe
res_mf.calculateTotalValue(df)

natural_gas_boiler  natural_gas  CH    3.237618e+05
                                 DE    1.894616e+06
dtype: float64

In [53]:
# and also get the full timeseries (disaggregate) if we want
df_disaggregate = res_mf.calculateFullTimeSeries(df)
df_disaggregate

natural_gas_boiler  natural_gas  CH  0         20.937711
                                     1         20.937711
                                     2         20.937711
                                     3         24.427713
                                     4         31.583558
                                                 ...    
                                 DE  26275    182.559451
                                     26276    182.559451
                                     26277    156.288835
                                     26278    114.550416
                                     26279    114.550416
Length: 52560, dtype: float64

### Some Plots

Here could be some example plots